<a href="https://colab.research.google.com/github/kuanlinhuang/decentralizedImmunizationEHR/blob/master/TradeTesla_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ==============================================================================
# PROTOTYPE: Multi-Agent System for Tesla (TSLA) Trading
#
# DISCLAIMER:
# This is a simplified educational prototype.
# It is NOT a real trading bot.
# It uses FAKE, RANDOMLY GENERATED data.
# DO NOT use this code for live trading. Trading is extremely risky.
# ==============================================================================

import time
import random
import requests # <-- Import the requests library
import yfinance as yf # <-- Import yfinance
import pandas as pd # <-- yfinance dependency
from collections import deque
from datetime import datetime, date, timedelta
import google.generativeai as genai
from google.colab import userdata

# We'll either use simulated time or real time
class TimeController:
    # _current_time = datetime(2025, 7, 27, 9, 30, 0)
    _current_time = datetime.now()

    @classmethod
    def now(cls):
        return cls._current_time

    @classmethod
    def advance_time(cls, seconds):
        cls._current_time += timedelta(seconds=seconds)

# Use TimeController.now() instead of datetime.now() throughout the simulation
now = TimeController.now

class EventBus:
    """A simple message bus for agent communication."""
    def __init__(self):
        self.listeners = {}

    def subscribe(self, event_type, listener_callback):
        """Register a listener for a specific event type."""
        if event_type not in self.listeners:
            self.listeners[event_type] = []
        self.listeners[event_type].append(listener_callback)
        # print(f"DEBUG: {listener_callback.__self__.__class__.__name__} subscribed to {event_type}")

    def publish(self, event_type, data):
        """Publish an event to all registered listeners."""
        if event_type in self.listeners:
            for callback in self.listeners[event_type]:
                callback(data)


In [17]:
# --- Agent Definitions ---

class BaseAgent:
    """Base class for all agents."""
    def __init__(self, event_bus):
        self.event_bus = event_bus

    def run_tick(self):
        """Each agent can have a main loop function, though most will be event-driven."""
        pass

In [18]:
class MarketDataAgent(BaseAgent):
    """Fetches real-time market data for TSLA using yfinance."""
    def __init__(self, event_bus, verbose=False):
        super().__init__(event_bus)
        self.symbol = "TSLA"
        self.ticker = yf.Ticker(self.symbol)
        self.last_published_price = None # Store the last published price
        self.verbose = verbose # Add verbose flag

    def run_tick(self):
        try:
            # Get the latest price data
            # Using history with a small period is a common way to get the latest price
            data = self.ticker.history(period="1d", interval="1m")

            if data.empty:
                if self.verbose:
                    print(f"⚠️  [MarketDataAgent]: No recent market data found for {self.symbol}.")
                return

            # Get the last closing price
            current_price = round(data['Close'].iloc[-1], 2)

            # Publish the first price or if the price change is significant
            if self.last_published_price is None or \
               abs(current_price - self.last_published_price) / self.last_published_price >= 0.005: # 0.5% change

                market_data = {
                    "symbol": self.symbol,
                    "price": current_price,
                    "timestamp": datetime.now() # Use the simulated time
                }

                print(f"[MarketDataAgent]: Publishing new price for {market_data['symbol']}: ${market_data['price']:.2f}{' (Initial Price)' if self.last_published_price is None else ' (Change >= 0.5%)'}")
                self.event_bus.publish("NEW_MARKET_DATA", market_data)
                self.last_published_price = current_price # Update the last published price
            else:
                if self.verbose:
                    print(f"[MarketDataAgent]: Price change less than 0.5%. Not publishing. Current price: ${current_price:.2f}")


        except Exception as e:
            print(f"⛔ [MarketDataAgent]: ERROR fetching market data from yfinance: {e}")

In [19]:
class ScheduledEventAgentFMP(BaseAgent):
    """Monitors an earnings schedule fetched from the FMP API."""
    def __init__(self, event_bus):
        super().__init__(event_bus)
        self.symbol = "TSLA"
        self.schedule = {}
        self.last_notified_event = None
        self.fetch_earnings_calendar() # Fetch data on initialization

    def fetch_earnings_calendar(self):
        """Pulls earnings data from FMP and populates the schedule."""
        print("[ScheduledEventAgent]: Fetching earnings calendar from FMP API...")

        FMP_API_KEY = userdata.get('FMP_API_KEY')
        genai.configure(api_key=FMP_API_KEY)
        print("✅ API Key configured successfully!")

        if FMP_API_KEY == "YOUR_API_KEY":
            print("⛔ [ScheduledEventAgent]: ERROR - Please set your FMP_API_KEY at the top of the script.")
            return

        url = f"https://financialmodelingprep.com/api/v3/earning_calendar?apikey={FMP_API_KEY}"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

            api_data = response.json()
            if not api_data:
                print("⚠️  [ScheduledEventAgent]: No earnings data returned from FMP.")
                return

            # Filter for our target symbol and populate the schedule
            for event in api_data:
                if event.get('symbol') == self.symbol:
                    event_date_str = event.get('date')
                    event_date = datetime.strptime(event_date_str, '%Y-%m-%d').date()
                    event_name = f"Earnings on {event_date_str}"
                    self.schedule[event_name] = event_date

            if self.schedule:
                 print(f"✅ [ScheduledEventAgent]: Successfully loaded schedule for {self.symbol}. Events found: {list(self.schedule.keys())}")
            else:
                 print(f"ℹ️  [ScheduledEventAgent]: No upcoming earnings for {self.symbol} found in the calendar.")

        except requests.exceptions.RequestException as e:
            print(f"⛔ [ScheduledEventAgent]: ERROR fetching data from FMP API: {e}")

    def run_tick(self):
        current_date = now().date()
        for event_name, event_date in self.schedule.items():
            if event_date < current_date: continue

            days_until = (event_date - current_date).days
            notification_key = f"{event_name}_{days_until}"

            if self.last_notified_event == notification_key: continue

            if days_until == 0:
                print(f"🚨 [ScheduledEventAgent]: EVENT IMMINENT! '{event_name}' is today for {self.symbol}.")
                self.event_bus.publish("EVENT_IMMINENT_FMP", {"symbol": self.symbol, "event_name": event_name})
                self.last_notified_event = notification_key
            elif 0 < days_until <= 3:
                print(f"⚠️  [ScheduledEventAgent]: UPCOMING EVENT! '{event_name}' in {days_until} days for {self.symbol}.")
                self.event_bus.publish("UPCOMING_EVENT_FMP", {"symbol": self.symbol, "event_name": event_name, "days_until": days_until})
                self.last_notified_event = notification_key

In [20]:
class ScheduledEventAgent(BaseAgent):
    """Monitors an earnings schedule fetched from Yahoo Finance."""
    def __init__(self, event_bus):
        super().__init__(event_bus)
        self.symbol = "TSLA"
        self.schedule = {}
        self.last_notified_event = None
        self.fetch_earnings_from_yfinance() # Fetch data on initialization

    def fetch_earnings_from_yfinance(self):
        """Pulls upcoming earnings data from Yahoo Finance."""
        print("[ScheduledEventAgent]: Fetching earnings calendar from Yahoo Finance...")
        try:
            tsla_ticker = yf.Ticker(self.symbol)
            # .get_earnings_dates() is more reliable for future dates
            earnings_dates = tsla_ticker.get_earnings_dates(limit=12) # Look at next 12 quarters

            if earnings_dates is None or earnings_dates.empty:
                print(f"⚠️  [ScheduledEventAgent]: No earnings data found for {self.symbol} via yfinance.")
                return

            # Filter for future dates
            # Make the simulated current time timezone-aware for comparison
            simulated_now_aware = pd.Timestamp(now(), tz='America/New_York')
            future_earnings = earnings_dates[earnings_dates.index > simulated_now_aware]

            for event_timestamp, row in future_earnings.iterrows():
                event_date = event_timestamp.date()
                event_name = f"Earnings on {event_date.strftime('%Y-%m-%d')}"
                self.schedule[event_name] = event_date

            if self.schedule:
                 print(f"✅ [ScheduledEventAgent]: Successfully loaded schedule for {self.symbol}. Events found: {list(self.schedule.keys())}")
            else:
                 print(f"ℹ️  [ScheduledEventAgent]: No upcoming earnings for {self.symbol} found via yfinance.")

        except Exception as e:
            print(f"⛔ [ScheduledEventAgent]: ERROR fetching data from Yahoo Finance: {e}")

    def run_tick(self):
        current_date = now().date()
        for event_name, event_date in self.schedule.items():
            if event_date < current_date: continue

            days_until = (event_date - current_date).days
            notification_key = f"{event_name}_{days_until}"

            if self.last_notified_event == notification_key: continue

            if days_until == 0:
                print(f"🚨 [ScheduledEventAgent]: EVENT IMMINENT! '{event_name}' is today for {self.symbol}.")
                self.event_bus.publish("EVENT_IMMINENT", {"symbol": self.symbol, "event_name": event_name})
                self.last_notified_event = notification_key
            elif 0 < days_until <= 3:
                print(f"⚠️  [ScheduledEventAgent]: UPCOMING EVENT! '{event_name}' in {days_until} days for {self.symbol}.")
                self.event_bus.publish("UPCOMING_EVENT", {"symbol": self.symbol, "event_name": event_name, "days_until": days_until})
                self.last_notified_event = notification_key

In [7]:
class SentimentAgent(BaseAgent):
    """Fetches news and analyzes its sentiment."""
    def __init__(self, event_bus, use_ollama=True, ollama_url="http://localhost:11434/api/generate", ollama_model="llama3.2:latest", ollama_username=None, ollama_password=None, verbose=False):
        super().__init__(event_bus)
        self.symbol = "TSLA"
        self.company_name = "Tesla" # Add company name for filtering
        self.news_cache = deque(maxlen=10) # Keep a small cache of recent news
        self.use_ollama = use_ollama
        self.ollama_url = ollama_url
        self.ollama_model = ollama_model
        self.ollama_username = ollama_username
        self.ollama_password = ollama_password
        self.verbose = verbose
        self.last_fetch_time = None
        self.fetch_interval = timedelta(hours=1) # Fetch news every hour (simulated)
        self.prompt_override = None # For testing purposes

    def fetch_news(self):
        """Fetches recent news for the symbol using yfinance."""
        print(f"[SentimentAgent]: Fetching news from Yahoo Finance...")
        try:
            # Fetch news using yfinance
            tsla_ticker = yf.Ticker(self.symbol)
            news = tsla_ticker.news

            if not news:
                print(f"⚠️  SentimentAgent: No news found for {self.symbol}.")
                return []

            print(f"✅ SentimentAgent: Found {len(news)} raw news items.")
            # Return all news items without explicit filtering
            return news

        except Exception as e:
            print(f"⛔ SentimentAgent: ERROR fetching news from Yahoo Finance: {e}")
            return []

    def _consolidate_news(self, news_items):
        """Consolidates a list of news items into a single string by attempting to get the main text content."""
        all_content_text = ""
        for item in news_items:
            # Attempt to get the main text content. Common keys might be 'text' or 'content'.
            # Falling back to 'summary' or 'title' if main content keys are not present.
            main_content = item.get('text') or item.get('content') or item.get('summary') or item.get('title')
            if main_content:
                all_content_text += f"{main_content}\n\n"
            else:
                all_content_text += "No main content available for this news item.\n\n"

        return all_content_text.strip()


    def analyze_sentiment(self, all_content_text):
        """Analyzes the sentiment of a given text using the configured model."""
        if self.verbose:
            print(f"[SentimentAgent]: Analyzing the following text:\n---\n{all_content_text}\n---")

        if self.use_ollama:
            print(f"[SentimentAgent]: Using Ollama for sentiment analysis.")
            return analyze_sentiment_with_ollama(
                all_content_text, # Pass the consolidated text
                self.ollama_url,
                self.ollama_model,
                self.ollama_username,
                self.ollama_password,
                prompt_override=self.prompt_override, # Pass the override
                symbol=self.symbol, # Pass symbol for prompt customization
                company_name=self.company_name # Pass company name for prompt customization
            )
        else:
            print(f"[SentimentAgent]: Using Gemini for sentiment analysis.")
            return analyze_sentiment_with_gemini(
                all_content_text,
                prompt_override=self.prompt_override,
                symbol=self.symbol,  # Pass symbol for prompt customization
                company_name=self.company_name # Pass company name for prompt customization
                )


    def run_tick(self):
        """Fetches news and analyzes sentiment periodically."""
        # Check if it's time to fetch new news or if prompt_override is set for a forced run
        if self.prompt_override is not None or \
           self.last_fetch_time is None or now() - self.last_fetch_time >= self.fetch_interval:

            # When prompt_override is used, we don't need to fetch real news.
            # Pass an empty list or a dummy list to the analyze_sentiment function.
            # The analyze_sentiment function should handle the prompt_override logic.
            if self.prompt_override is not None:
                # If prompt_override is used, we don't fetch news,
                # so we don't have news_items to consolidate.
                # The analyze_sentiment function should handle the prompt_override directly.
                all_content_text = "" # Provide an empty string when overriding
                print("[SentimentAgent]: Using prompt_override. Skipping news fetch and consolidation.")
            else:
                news_items = self.fetch_news()
                self.last_fetch_time = now() # Update fetch time only on actual fetch

                if not news_items:
                    print("[SentimentAgent]: No news found to analyze.")
                    # Publish a neutral sentiment if no relevant news is found
                    # The sentiment analysis function will handle relevance in the prompt
                    self.event_bus.publish("SENTIMENT_SIGNAL", {
                        "symbol": self.symbol,
                        "sentiment": "NEUTRAL",
                        "source": "SentimentAgent", # Indicate source is the agent itself
                        "headline": "No news found" # Update headline
                    })
                    return # No news to process

                # Consolidate all news items into a single string
                all_content_text = self._consolidate_news(news_items)


            # Analyze overall sentiment for all news items (or use prompt_override) and publish
            print(f"[SentimentAgent]: Analyzing overall sentiment.")
            sentiment_result = self.analyze_sentiment(all_content_text) # Pass the consolidated text
            print(f"[SentimentAgent]: Overall sentiment analysis complete -> Publishing sentiment: {sentiment_result['sentiment']} from {sentiment_result['source']}")
            self.event_bus.publish("SENTIMENT_SIGNAL", sentiment_result)

            # Reset override after use if it was set
            if self.prompt_override is not None:
                self.prompt_override = None

In [8]:
# analyze_sentiment_with_ollama function definition
def analyze_sentiment_with_ollama(all_content_text, ollama_url, ollama_model="llama3.2:latest", username=None, password=None, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using Ollama."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Ollama Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        prompt = f"""Analyze the overall sentiment of the following financial news headlines and summaries about Tesla ({symbol}). Categorize the overall sentiment as POSITIVE, NEUTRAL, or NEGATIVE.
Respond with only one word: POSITIVE, NEUTRAL, or NEGATIVE.

News Content:
---
{all_content_text}
---

Overall Sentiment:"""
        print(f"ℹ️  Ollama Client: Using default prompt for overall sentiment analysis.")


    try:
        # Make a real HTTP POST request to your Ollama instance
        data = {
            "model": ollama_model,
            "prompt": prompt,
            "stream": False # We want the full response at once
        }

        # Use basic authentication if username and password are provided
        auth = (username, password) if username and password else None

        # Explicitly set the Origin header
        # We'll use the base ngrok URL as the origin
        origin_url = ollama_url.replace("/api/generate", "") if "/api/generate" in ollama_url else ollama_url
        headers = {
            "Origin": origin_url
        }

        # --- Add print statements to check username and password ---
        print(f"ℹ️  Ollama Client: Authenticating with Username: {username}, Password provided: {'Yes' if password else 'No'}")
        # --- End print statements ---

        print(f"ℹ️  Ollama Client: Sending sentiment analysis request to Ollama.")
        # Pass the auth parameter to requests.post
        response = requests.post(ollama_url, json=data, headers=headers, auth=auth) # Pass headers and auth
        print(f"ℹ️  Ollama Client: POST request sent. Status Code: {response.status_code}") # Debug print
        response.raise_for_status() # Raise an exception for bad status codes
        result = response.json()
        print(f"ℹ️  Ollama Client: Received response from Ollama.") # Debug print
        sentiment = result.get('response', '').strip().upper()

        # Basic validation
        if sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
            print(f"✅ Ollama Client: Ollama sentiment: {sentiment}")
            return {
                "symbol": symbol, # Add symbol here
                "sentiment": sentiment,
                "source": "Ollama",
                "headline": "Overall News Sentiment" # Indicate this is overall sentiment
            }
        else:
            print(f"⚠️  Ollama Client: Ollama returned unexpected sentiment format: '{sentiment}'. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol, # Add symbol here
                "sentiment": "NEUTRAL",
                "source": "Ollama",
                "headline": "Overall News Sentiment" # Indicate this is overall sentiment
            }

    except requests.exceptions.RequestException as e:
        print(f"⛔ Ollama Client: ERROR communicating with Ollama instance at {ollama_url}: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment" # Indicate this is overall sentiment
        }
    except Exception as e:
        print(f"⛔ Ollama Client: An unexpected error occurred during Ollama analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment" # Indicate this is overall sentiment
        }






def analyze_sentiment_with_gemini(all_content_text, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using the Gemini API."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Gemini Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        prompt = f"""Analyze the overall sentiment of the following financial news headlines and summaries about Tesla ({symbol}). Categorize the overall sentiment as POSITIVE, NEUTRAL, or NEUTRAL.
Respond with only one word: POSITIVE, NEUTRAL, or NEUTRAL.

News Content:
---
{all_content_text}
---

Overall Sentiment:"""
        print(f"ℹ️  Gemini Client: Using default prompt for overall sentiment analysis.")

    try:
        # Configure Gemini API
        # Assumes GOOGLE_API_KEY is stored in Colab secrets
        print("ℹ️  Gemini Client: Attempting to access GOOGLE_API_KEY from Colab secrets.")
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)

        # Initialize the Gemini model
        # Use a suitable model, e.g., 'gemini-1.5-flash-latest' or 'gemini-1.0-pro'
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # You can change the model here

        print(f"ℹ️  Gemini Client: Sending sentiment analysis request to Gemini.")

        # Generate content using the prompt
        response = gemini_model.generate_content(prompt)

        sentiment = response.text.strip().upper()

        # Basic validation
        if sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
            print(f"✅ Gemini Client: Gemini sentiment: {sentiment}")
            return {
                "symbol": symbol, # Add symbol here
                "sentiment": sentiment,
                "source": "Gemini",
                "headline": "Overall News Sentiment" # Indicate this is overall sentiment
            }
        else:
            print(f"⚠️  Gemini Client: Gemini returned unexpected sentiment format: '{sentiment}'. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol, # Add symbol here
                "sentiment": "NEUTRAL",
                "source": "Gemini",
                "headline": "Overall News Sentiment" # Indicate this is overall sentiment
            }

    except TimeoutError as e:
        print(f"⛔ Gemini Client: ERROR: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI. Please ensure you are running directly in the Colab UI. Error: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment" # Indicate this is overall sentiment
        }
    except Exception as e:
        print(f"⛔ Gemini Client: An unexpected error occurred during Gemini analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment" # Indicate this is overall sentiment
        }

# SentimentAgent class definition is in a separate cell (cell fac0357d)

In [9]:
class TechnicalAnalysisAgent(BaseAgent):
    """Performs technical analysis on market data."""
    def __init__(self, event_bus, short_window=5, long_window=12):
        super().__init__(event_bus)
        self.short_window = short_window
        self.long_window = long_window
        self.prices = deque(maxlen=self.long_window)
        self.last_signal = None
        self.event_bus.subscribe("NEW_MARKET_DATA", self.handle_new_data)

    def handle_new_data(self, data):
        self.prices.append(data['price'])

        if len(self.prices) < self.long_window:
            return # Not enough data yet

        short_sma = sum(list(self.prices)[-self.short_window:]) / self.short_window
        long_sma = sum(self.prices) / self.long_window

        current_signal = None
        if short_sma > long_sma:
            current_signal = "GOLDEN_CROSS" # Bullish
        else:
            current_signal = "DEATH_CROSS"  # Bearish

        # Publish only if the signal changes to avoid spamming
        if current_signal != self.last_signal:
            self.last_signal = current_signal
            signal_data = {"symbol": "TSLA", "signal": current_signal, "short_sma": short_sma, "long_sma": long_sma}
            print(f"[TA_Agent]: New signal detected: {signal_data['signal']} (Short SMA: {short_sma:.2f}, Long SMA: {long_sma:.2f})")
            self.event_bus.publish("TA_SIGNAL", signal_data)

In [10]:
class StrategyAgent(BaseAgent):
    """Generates trading proposals based on signals."""
    def __init__(self, event_bus):
        super().__init__(event_bus)
        self.last_ta_signal = None
        self.last_sentiment = None
        self.event_bus.subscribe("TA_SIGNAL", self.handle_ta_signal)
        self.event_bus.subscribe("SENTIMENT_SIGNAL", self.handle_sentiment_signal)

    def handle_ta_signal(self, data):
        self.last_ta_signal = data['signal']
        self.evaluate_strategy()

    def handle_sentiment_signal(self, data):
        self.last_sentiment = data['sentiment']
        self.evaluate_strategy()

    def evaluate_strategy(self):
        # Only evaluate if we have fresh data for both
        if not self.last_ta_signal or not self.last_sentiment:
            return

        proposal = None
        # --- Bullish Strategy ---
        if self.last_ta_signal == "GOLDEN_CROSS" and self.last_sentiment == "POSITIVE":
            proposal = {"symbol": "TSLA", "action": "BUY", "quantity": 10, "reason": "Golden Cross + Positive Sentiment"}

        # --- Bearish Strategy ---
        elif self.last_ta_signal == "DEATH_CROSS" and self.last_sentiment == "NEGATIVE":
            proposal = {"symbol": "TSLA", "action": "SELL", "quantity": 10, "reason": "Death Cross + Negative Sentiment"}

        if proposal:
            print(f"[StrategyAgent]: PROPOSING TRADE -> {proposal['action']} {proposal['quantity']} {proposal['symbol']} ({proposal['reason']})")
            self.event_bus.publish("TRADE_PROPOSAL", proposal)

        # Reset signals to wait for new ones
        self.last_ta_signal = None
        self.last_sentiment = None



In [11]:
class RiskManagementAgent(BaseAgent):
    """Approves or rejects trades based on risk rules."""
    def __init__(self, event_bus):
        super().__init__(event_bus)
        self.position_size = 0
        self.max_position = 20
        self.safe_mode_enabled = False # New property for event risk
        self.event_bus.subscribe("TRADE_PROPOSAL", self.evaluate_risk)
        self.event_bus.subscribe("TRADE_EXECUTED", self.update_position)
        self.event_bus.subscribe("EVENT_IMMINENT", self.enable_safe_mode)

    def enable_safe_mode(self, data):
        print(f"🔒 [RiskAgent]: SAFE MODE ENABLED due to imminent event: {data['event_name']}. No new positions will be opened.")
        self.safe_mode_enabled = True

    def evaluate_risk(self, proposal):
        # Rule 1: Event Risk - Reject any new positions if safe mode is on
        if self.safe_mode_enabled and (proposal['action'] == "BUY"):
            print(f"⛔ [RiskAgent]: REJECTED. Reason: Safe mode is active due to imminent earnings.")
            return

        # Rule 2: Position Sizing
        if proposal['action'] == "BUY" and (self.position_size + proposal['quantity']) > self.max_position:
            print(f"⛔ [RiskAgent]: REJECTED. Reason: Exceeds max position size.")
            return

        # Rule 3: No Shorting Logic
        if proposal['action'] == "SELL" and self.position_size <= 0:
            print(f"⛔ [RiskAgent]: REJECTED. Reason: No long position to sell.")
            return

        print(f"✅ [RiskAgent]: Trade APPROVED. Current position: {self.position_size}")
        self.event_bus.publish("TRADE_APPROVED", proposal)

    def update_position(self, executed_trade):
        if executed_trade['action'] == "BUY":
            self.position_size += executed_trade['quantity']
        elif executed_trade['action'] == "SELL":
            self.position_size -= executed_trade['quantity']
        print(f"[RiskAgent]: Position updated. New position size: {self.position_size}")

In [12]:
class ExecutionAgent(BaseAgent):
    """Simulates executing trades with a broker."""
    def __init__(self, event_bus):
        super().__init__(event_bus)
        self.event_bus.subscribe("TRADE_APPROVED", self.execute_trade)

    def execute_trade(self, trade_details):
        print("\n" + "="*25)
        print("!!! EXECUTION AGENT !!!")
        print(f"--- Sending {trade_details['action']} order for {trade_details['quantity']} shares of {trade_details['symbol']} to broker. ---")
        print("="*25 + "\n")
        # In a real system, you'd get a confirmation from the broker
        self.event_bus.publish("TRADE_EXECUTED", trade_details)

In [15]:
# Assuming all the agent classes from the prototype are defined above

# Added prompt_override as an explicit argument to test_publisher
def test_publisher(Bus, Agent, Topic, prompt_override=None, *agent_args, **agent_kwargs):
    print("--- Testing ---")

    # This list will capture any data our agent publishes
    captured_data = []
    captured_all_content_text = "" # Variable to store the consolidated text

    # A simple listener function
    def data_listener(data):
        nonlocal captured_all_content_text # Allow modification of the outer scope variable
        print(f"Listener caught data: {data}")
        captured_data.append(data)
        # Assuming the all_content_text is part of the data published,
        # or we need to access it from the agent instance.
        # Since the agent analyzes and publishes, the consolidated text is
        # generated before publishing. We need to capture it before it's used
        # in the analyze_sentiment call within run_tick.

    # Subscribe our listener to the event the agent publishes
    Bus.subscribe(Topic, data_listener)

    # Create the agent, passing through other args/kwargs for initialization
    agent = Agent(Bus, *agent_args, **agent_kwargs)

    # Temporarily set the prompt_override attribute on the agent instance for this test run
    # The agent's run_tick method will need to check for this attribute
    if prompt_override is not None:
        agent.prompt_override = prompt_override

    # Modify the run_tick method of the agent instance for testing purposes
    # to capture the all_content_text before analysis and publishing
    original_run_tick = agent.run_tick
    def modified_run_tick():
        nonlocal captured_all_content_text
        # Call the original run_tick
        original_run_tick()
        # After the original run_tick, the agent will have processed news and
        # potentially analyzed sentiment. We need to find a way to access
        # the all_content_text that was generated just before analysis.
        # This is tricky because the agent's internal state might not
        # expose this easily.

        # A simpler approach for testing is to modify the analyze_sentiment call
        # within run_tick to also print the all_content_text.
        pass # We will modify analyze_sentiment instead

    # Modify the analyze_sentiment method of the agent instance
    original_analyze_sentiment = agent.analyze_sentiment
    def modified_analyze_sentiment(all_content_text_arg):
        print(f"--- Consolidated News Text for Analysis ---\n{all_content_text_arg}\n--- End Consolidated News Text ---")
        # Call the original analyze_sentiment
        return original_analyze_sentiment(all_content_text_arg)

    agent.analyze_sentiment = modified_analyze_sentiment


    # Call the agent's main tick function
    agent.run_tick()

    # Verify the results
    # We expect at least one sentiment signal to be published if news is found
    assert len(captured_data) > 0, "Expected at least one sentiment signal, but got none."
    assert captured_data[0]['symbol'] == 'TSLA'
    # Optional: Add checks for sentiment or headline based on prompt_override if applicable
    print("✅ Test Passed: The Agent published its data correctly.\n")

# Run the test
Bus = EventBus()
# --- Update this line to pass your ngrok URL ---
# Replace "YOUR_NGROK_HTTPS_URL" with the https:// URL from ngrok + /api/generate
ngrok_ollama_url = "https://47c47f70c36c.ngrok-free.app/api/generate" # <-- PASTE YOUR CORRECT NGROK URL + /api/generate HERE
username = "hw0531" # <-- Your username here
password = "Aol12345" # <-- Your password here

#Agent = MarketDataAgent(Bus)
#Topic = "NEW_MARKET_DATA"
#Agent = ScheduledEventAgent(Bus)
#Topic = "EVENT_IMMINENT"
Agent = SentimentAgent
Topic = "SENTIMENT_SIGNAL"

# Example of calling test_publisher with prompt_override (replace the string as needed)
test_publisher(
    Bus,
    Agent,
    Topic,
    # prompt_override="Analyze the sentiment of this text: 'TSLA stock is plumetting!' Respond with one word: POSITIVE, NEUTRAL, or NEGATIVE.",
    ollama_url=ngrok_ollama_url,
    use_ollama=True, # Set to False to test Gemini
    ollama_username=username,
    ollama_password=password,
    verbose=False # Set to True for verbose output
)

--- Testing ---
[SentimentAgent]: Fetching news from Yahoo Finance...
✅ SentimentAgent: Found 10 raw news items.
[SentimentAgent]: Analyzing overall sentiment.
--- Consolidated News Text for Analysis ---
{'id': 'd55926b5-00fb-49a2-8aa9-f04f70e956fe', 'contentType': 'STORY', 'title': "Tesla's Europe sales are weak again in July, but bright spots emerge", 'description': '', 'summary': "Tesla's sales hangover rolled on in certain key territories in Europe, as the introduction of the revamped Model Y wasn’t enough to blunt the effect of rising competition and CEO Elon Musk's deep unpopularity. However, there were a couple of bright spots for the beleaguered EV maker.", 'pubDate': '2025-08-01T13:32:55Z', 'displayTime': '2025-08-01T13:32:55Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-06/027dd590-4baa-11f0-b9fd-52929405488c', 'originalWidth': 7878, 'originalHeight': 5252, 'caption': '', 'resolut

TypeError: analyze_sentiment_with_ollama() got an unexpected keyword argument 'symbol'

In [ ]:
# Assuming all the agent classes from the prototype are defined above

def test_subscriber(Bus, Agent):
    print("--- Testing TechnicalAnalysisAgent ---")

    # This list will capture the signal from our TA agent
    captured_signals = []

    def signal_listener(data):
        print(f"Listener caught signal: {data}")
        captured_signals.append(data)

    # Subscribe our listener to the TA agent's output event
    Bus.subscribe("TA_SIGNAL", signal_listener)

    # Create the agent. It automatically subscribes to "NEW_MARKET_DATA".
    # Use smaller windows for easier testing.
    ta_agent = Agent

    # Manually publish the data it needs. We'll create a "Golden Cross".
    # Prices are low and then start rising.
    # Provide enough initial data to meet the long_window requirement (default is 12)
    initial_low_prices = [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311]
    rising_prices = [312, 315, 320, 325, 330]

    print("Publishing initial low prices to fill the buffer...")
    for price in initial_low_prices:
        Bus.publish("NEW_MARKET_DATA", {"price": price})

    print("Publishing rising prices to trigger a cross...")
    for price in rising_prices:
        Bus.publish("NEW_MARKET_DATA", {"price": price})


    # Verify the results
    # We expect at least one signal after publishing all data, and the last one should be GOLDEN_CROSS
    assert len(captured_signals) > 0, "Expected at least one signal, but got none."
    assert captured_signals[-1]['signal'] == 'GOLDEN_CROSS', f"Expected last signal to be 'GOLDEN_CROSS', but got '{captured_signals[-1]['signal']}'"
    print("✅ Test Passed: TechnicalAnalysisAgent correctly identified a Golden Cross.\n")

# Run the test
Bus = EventBus()
Agent = TechnicalAnalysisAgent(Bus) # Use default windows
#Topic = "NEW_MARKET_DATA"
test_subscriber(Bus, Agent)

--- Testing TechnicalAnalysisAgent ---
Publishing initial low prices to fill the buffer...
[TA_Agent]: New signal detected: GOLDEN_CROSS (Short SMA: 309.00, Long SMA: 305.50)
Listener caught signal: {'symbol': 'TSLA', 'signal': 'GOLDEN_CROSS', 'short_sma': 309.0, 'long_sma': 305.5}
Publishing rising prices to trigger a cross...
✅ Test Passed: TechnicalAnalysisAgent correctly identified a Golden Cross.



In [ ]:
# --- Main Simulation Loop ---

if __name__ == "__main__":
    bus = EventBus()

    # Create all the agents
    market_data_agent = MarketDataAgent(bus) # Assign to a variable
    scheduled_event_agent = ScheduledEventAgent(bus) # Assign to a variable
    sentiment_agent = SentimentAgent(bus) # Assign to a variable
    technical_analysis_agent = TechnicalAnalysisAgent(bus) # Assign to a variable
    strategy_agent = StrategyAgent(bus) # Assign to a variable
    risk_management_agent = RiskManagementAgent(bus) # Assign to a variable
    execution_agent = ExecutionAgent(bus) # Assign to a variable


    agents = [
        market_data_agent,
        scheduled_event_agent,
        sentiment_agent,
        technical_analysis_agent,
        strategy_agent,
        risk_management_agent,
        execution_agent
    ]

    print("--- Starting Multi-Agent Trading Simulation (V2) ---")
    print(f"--- Simulation Starting Date: {now().strftime('%Y-%m-%d')} (Press Ctrl+C to stop) ---\n")
    tick_interval = 2 # seconds

    try:
        # Initial run for agents that should start immediately
        market_data_agent.run_tick() # Get initial market data
        scheduled_event_agent.run_tick() # Check for initial events
        sentiment_agent.run_tick() # Get initial sentiment
        #TechnicalAnalysisAgent needs data before running, it's triggered by NEW_MARKET_DATA

        while True:
            # Print the current simulated date/time at the start of each day's simulation
            print(f"--- Tick at {now().strftime('%Y-%m-%d %H:%M:%S')} ---")

            # Run agents that need to check state periodically or are not purely event-driven
            scheduled_event_agent.run_tick()
            sentiment_agent.run_tick()
            # MarketDataAgent is now event-driven based on price change

            print("-" * 75)
            # Advance the controlled time. Let's say each tick represents 4 hours.
            TimeController.advance_time(seconds=4 * 3600)
            time.sleep(tick_interval)

    except KeyboardInterrupt:
        print("\n--- Simulation stopped by user. ---")

[ScheduledEventAgent]: Fetching earnings calendar from Yahoo Finance...
ℹ️  [ScheduledEventAgent]: No upcoming earnings for TSLA found via yfinance.
--- Starting Multi-Agent Trading Simulation (V2) ---
--- Simulation Starting Date: 2025-07-31 (Press Ctrl+C to stop) ---

[MarketDataAgent]: Publishing new price for TSLA: $308.19 (Initial Price)
[SentimentAgent]: Fetching news from Yahoo Finance...
✅ SentimentAgent: Found 10 news items.
[SentimentAgent]: Sending news item for analysis: 'Apple quietens Wall Street’s fears of China struggles and slow AI progress'
[SentimentAgent]: Using Ollama for sentiment analysis.
ℹ️  Ollama Client: Sending sentiment analysis request to Ollama for: 'Apple quietens Wall Street’s fears of China struggles and slow AI progress'
⛔ Ollama Client: ERROR communicating with Ollama instance at http://localhost:11434/api/generate: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.c

In [ ]:
# Remove existing Ollama installation
!ollama stop || true # Stop any running server gracefully if possible
!rm -rf /usr/local/bin/ollama
!rm -rf /usr/local/share/ollama
!rm -rf ~/.ollama # Remove models and other user data
print("Existing Ollama installation removed.")

/bin/bash: line 1: ollama: command not found
Existing Ollama installation removed.


In [ ]:
# Install Ollama in the Colab VM
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


# Task
Modify the code to include the percentage of positive and negative news in the sentiment analysis output.

## Modify language model prompts

### Subtask:
Update the prompts in `analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini` to request a more detailed sentiment breakdown, including percentages or counts of positive/negative news items.


**Reasoning**:
The subtask requires modifying the prompts within the `analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini` functions to request a detailed sentiment breakdown. This involves updating the prompt strings within the existing function definitions.



In [14]:
# analyze_sentiment_with_ollama function definition
def analyze_sentiment_with_ollama(all_content_text, ollama_url, ollama_model="llama3.2:latest", username=None, password=None, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using Ollama."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Ollama Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about Tesla ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEGATIVE. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Ollama Client: Using default prompt for detailed sentiment analysis.")


    try:
        # Make a real HTTP POST request to your Ollama instance
        data = {
            "model": ollama_model,
            "prompt": prompt,
            "stream": False # We want the full response at once
        }

        # Use basic authentication if username and password are provided
        auth = (username, password) if username and password else None

        # Explicitly set the Origin header
        # We'll use the base ngrok URL as the origin
        origin_url = ollama_url.replace("/api/generate", "") if "/api/generate" in ollama_url else ollama_url
        headers = {
            "Origin": origin_url
        }

        print(f"ℹ️  Ollama Client: Authenticating with Username: {username}, Password provided: {'Yes' if password else 'No'}")

        print(f"ℹ️  Ollama Client: Sending sentiment analysis request to Ollama.")
        # Pass the auth parameter to requests.post
        response = requests.post(ollama_url, json=data, headers=headers, auth=auth) # Pass headers and auth
        print(f"ℹ️  Ollama Client: POST request sent. Status Code: {response.status_code}") # Debug print
        response.raise_for_status() # Raise an exception for bad status codes
        result = response.json()
        print(f"ℹ️  Ollama Client: Received response from Ollama.") # Debug print

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(result.get('response', '').strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Ollama Client: Ollama sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Ollama",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Ollama Client: Ollama returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Ollama",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Ollama Client: Ollama did not return valid JSON. Raw response: {result.get('response', '')}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Ollama",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except requests.exceptions.RequestException as e:
        print(f"⛔ Ollama Client: ERROR communicating with Ollama instance at {ollama_url}: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Ollama Client: An unexpected error occurred during Ollama analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }






def analyze_sentiment_with_gemini(all_content_text, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using the Gemini API."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Gemini Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about Tesla ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEGATIVE. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Gemini Client: Using default prompt for detailed sentiment analysis.")


    try:
        # Configure Gemini API
        # Assumes GOOGLE_API_KEY is stored in Colab secrets
        print("ℹ️  Gemini Client: Attempting to access GOOGLE_API_KEY from Colab secrets.")
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)

        # Initialize the Gemini model
        # Use a suitable model, e.g., 'gemini-1.5-flash-latest' or 'gemini-1.0-pro'
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # You can change the model here

        print(f"ℹ️  Gemini Client: Sending sentiment analysis request to Gemini.")

        # Generate content using the prompt
        response = gemini_model.generate_content(prompt)

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(response.text.strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Gemini Client: Gemini sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Gemini",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Gemini Client: Gemini returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Gemini",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Gemini Client: Gemini did not return valid JSON. Raw response: {response.text.strip()}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Gemini",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except TimeoutError as e:
        print(f"⛔ Gemini Client: ERROR: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI. Please ensure you are running directly in the Colab UI. Error: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Gemini Client: An unexpected error occurred during Gemini analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }

# SentimentAgent class definition is in a separate cell (cell fac0357d)
import json # Import json for parsing the response

## Update sentiment analysis functions

### Subtask:
Modify `analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini` to parse the more detailed response from the language models and extract the positive and negative percentages.


**Reasoning**:
Modify the sentiment analysis functions to parse the JSON response and include the breakdown in the returned dictionary.



In [47]:
# analyze_sentiment_with_ollama function definition
def analyze_sentiment_with_ollama(all_content_text, ollama_url, ollama_model="llama3.2:latest", username=None, password=None, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using Ollama."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Ollama Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about Tesla ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEGATIVE. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Ollama Client: Using default prompt for detailed sentiment analysis.")


    try:
        # Make a real HTTP POST request to your Ollama instance
        data = {
            "model": ollama_model,
            "prompt": prompt,
            "stream": False # We want the full response at once
        }

        # Use basic authentication if username and password are provided
        auth = (username, password) if username and password else None

        # Explicitly set the Origin header
        # We'll use the base ngrok URL as the origin
        origin_url = ollama_url.replace("/api/generate", "") if "/api/generate" in ollama_url else ollama_url
        headers = {
            "Origin": origin_url
        }

        print(f"ℹ️  Ollama Client: Authenticating with Username: {username}, Password provided: {'Yes' if password else 'No'}")

        print(f"ℹ️  Ollama Client: Sending sentiment analysis request to Ollama.")
        # Pass the auth parameter to requests.post
        response = requests.post(ollama_url, json=data, headers=headers, auth=auth) # Pass headers and auth
        print(f"ℹ️  Ollama Client: POST request sent. Status Code: {response.status_code}") # Debug print
        response.raise_for_status() # Raise an exception for bad status codes
        result = response.json()
        print(f"ℹ️  Ollama Client: Received response from Ollama.") # Debug print

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(result.get('response', '').strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Ollama Client: Ollama sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Ollama",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Ollama Client: Ollama returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Ollama",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Ollama Client: Ollama did not return valid JSON. Raw response: {result.get('response', '')}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Ollama",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except requests.exceptions.RequestException as e:
        print(f"⛔ Ollama Client: ERROR communicating with Ollama instance at {ollama_url}: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Ollama Client: An unexpected error occurred during Ollama analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }






def analyze_sentiment_with_gemini(all_content_text, prompt_override=None):
    """Analyzes the sentiment of a given news item or list of news items using the Gemini API."""
    symbol = "TSLA" # Assuming TSLA for this prototype

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Gemini Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about Tesla ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEGATIVE. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Gemini Client: Using default prompt for detailed sentiment analysis.")


    try:
        # Configure Gemini API
        # Assumes GOOGLE_API_KEY is stored in Colab secrets
        print("ℹ️  Gemini Client: Attempting to access GOOGLE_API_KEY from Colab secrets.")
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)

        # Initialize the Gemini model
        # Use a suitable model, e.g., 'gemini-1.5-flash-latest' or 'gemini-1.0-pro'
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # You can change the model here

        print(f"ℹ️  Gemini Client: Sending sentiment analysis request to Gemini.")

        # Generate content using the prompt
        response = gemini_model.generate_content(prompt)

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(response.text.strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Gemini Client: Gemini sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Gemini",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Gemini Client: Gemini returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Gemini",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Gemini Client: Gemini did not return valid JSON. Raw response: {response.text.strip()}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Gemini",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except TimeoutError as e:
        print(f"⛔ Gemini Client: ERROR: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI. Please ensure you are running directly in the Colab UI. Error: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Gemini Client: An unexpected error occurred during Gemini analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }

# SentimentAgent class definition is in a separate cell (cell fac0357d)
import json # Import json for parsing the response

## Sentimentagent

### Subtask:
Adjust the `SentimentAgent` to handle the new sentiment data structure if necessary (e.g., if it expects just a single sentiment category).


## Update strategyagent

### Subtask:
If the `StrategyAgent` uses the sentiment for decision making, assess if it needs to be updated to utilize the percentage data.


## Test the changes

### Subtask:
Run the `test_publisher` and potentially the main simulation loop to ensure the sentiment percentages are correctly calculated and published.


**Reasoning**:
The subtask is to run the `test_publisher` and verify the output to ensure the sentiment percentages are included in the published data. The `test_publisher` function and its call are in cell `6Xt6lVKah7YG`. I will execute this cell.



In [48]:
# Assuming all the agent classes from the prototype are defined above

# Added prompt_override as an explicit argument to test_publisher
def test_publisher(Bus, Agent, Topic, prompt_override=None, *agent_args, **agent_kwargs):
    print("--- Testing ---")

    # This list will capture any data our agent publishes
    captured_data = []
    captured_all_content_text = "" # Variable to store the consolidated text

    # A simple listener function
    def data_listener(data):
        nonlocal captured_all_content_text # Allow modification of the outer scope variable
        print(f"Listener caught data: {data}")
        captured_data.append(data)
        # Assuming the all_content_text is part of the data published,
        # or we need to access it from the agent instance.
        # Since the agent analyzes and publishes, the consolidated text is
        # generated before publishing. We need to capture it before it's used
        # in the analyze_sentiment call within run_tick.

    # Subscribe our listener to the event the agent publishes
    Bus.subscribe(Topic, data_listener)

    # Create the agent, passing through other args/kwargs for initialization
    agent = Agent(Bus, *agent_args, **agent_kwargs)

    # Temporarily set the prompt_override attribute on the agent instance for this test run
    # The agent's run_tick method will need to check for this attribute
    if prompt_override is not None:
        agent.prompt_override = prompt_override

    # Modify the run_tick method of the agent instance for testing purposes
    # to capture the all_content_text before analysis and publishing
    original_run_tick = agent.run_tick
    def modified_run_tick():
        nonlocal captured_all_content_text
        # Call the original run_tick
        original_run_tick()
        # After the original run_tick, the agent will have processed news and
        # potentially analyzed sentiment. We need to find a way to access
        # the all_content_text that was generated just before analysis.
        # This is tricky because the agent's internal state might not
        # expose this easily.

        # A simpler approach for testing is to modify the analyze_sentiment call
        # within run_tick to also print the all_content_text.
        pass # We will modify analyze_sentiment instead

    # Modify the analyze_sentiment method of the agent instance
    original_analyze_sentiment = agent.analyze_sentiment
    def modified_analyze_sentiment(all_content_text_arg):
        print(f"--- Consolidated News Text for Analysis ---\n{all_content_text_arg}\n--- End Consolidated News Text ---")
        # Call the original analyze_sentiment
        return original_analyze_sentiment(all_content_text_arg)

    agent.analyze_sentiment = modified_analyze_sentiment


    # Call the agent's main tick function
    agent.run_tick()

    # Verify the results
    # We expect at least one sentiment signal to be published if news is found
    assert len(captured_data) > 0, "Expected at least one sentiment signal, but got none."
    assert captured_data[0]['symbol'] == 'TSLA'
    # Optional: Add checks for sentiment or headline based on prompt_override if applicable
    print("✅ Test Passed: The Agent published its data correctly.\n")

# Run the test
Bus = EventBus()
# --- Update this line to pass your ngrok URL ---
# Replace "YOUR_NGROK_HTTPS_URL" with the https:// URL from ngrok + /api/generate
ngrok_ollama_url = "https://47c47f70c36c.ngrok-free.app/api/generate" # <-- PASTE YOUR CORRECT NGROK URL + /api/generate HERE
username = "hw0531" # <-- Your username here
password = "Aol12345" # <-- Your password here

#Agent = MarketDataAgent(Bus)
#Topic = "NEW_MARKET_DATA"
#Agent = ScheduledEventAgent(Bus)
#Topic = "EVENT_IMMINENT"
Agent = SentimentAgent
Topic = "SENTIMENT_SIGNAL"

# Example of calling test_publisher with prompt_override (replace the string as needed)
test_publisher(
    Bus,
    Agent,
    Topic,
    # prompt_override="Analyze the sentiment of this text: 'TSLA stock is plumetting!' Respond with one word: POSITIVE, NEUTRAL, or NEGATIVE.",
    ollama_url=ngrok_ollama_url,
    use_ollama=True, # Set to False to test Gemini
    ollama_username=username,
    ollama_password=password,
    verbose=True # Set to True for verbose output
)

--- Testing ---
[SentimentAgent]: Fetching news from Yahoo Finance...
✅ SentimentAgent: Found 10 raw news items.
[SentimentAgent]: Analyzing overall sentiment.
--- Consolidated News Text for Analysis ---
{'id': 'b7c0fe9a-ad27-366c-8a8b-ebfe5a980438', 'contentType': 'STORY', 'title': 'Tesla sales in France and Denmark continue downward trend', 'description': '', 'summary': 'Investing.com -- Tesla Inc (NASDAQ:TSLA) car sales in France and Denmark declined in July for the seventh consecutive month.', 'pubDate': '2025-08-01T07:56:00Z', 'displayTime': '2025-08-01T07:56:00Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': None, 'provider': {'displayName': 'Investing.com', 'url': 'https://www.investing.com/'}, 'canonicalUrl': {'url': 'https://finance.yahoo.com/news/tesla-sales-france-denmark-continue-075600825.html', 'site': 'finance', 'region': 'US', 'lang': 'en-US'}, 'clickThroughUrl': {'url': 'https://finance.yahoo.com/news/tesla-sales-france-denmark-continue-0756

TypeError: analyze_sentiment_with_ollama() got an unexpected keyword argument 'symbol'

**Reasoning**:
The traceback indicates that the `analyze_sentiment_with_ollama` function received an unexpected keyword argument 'symbol'. Looking at the SentimentAgent's `analyze_sentiment` method, it is indeed passing `symbol=self.symbol` and `company_name=self.company_name` to both `analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini`. However, the current definition of `analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini` in cell `6Wt01en5JSow` does not accept these arguments. I need to modify the function signatures in cell `6Wt01en5JSow` to accept these arguments and include the updated code in a code block.



In [49]:
# analyze_sentiment_with_ollama function definition
def analyze_sentiment_with_ollama(all_content_text, ollama_url, ollama_model="llama3.2:latest", username=None, password=None, prompt_override=None, symbol="TSLA", company_name="Tesla"):
    """Analyzes the sentiment of a given news item or list of news items using Ollama."""
    # symbol and company_name are now parameters with default values

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Ollama Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about {company_name} ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEGATIVE. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Ollama Client: Using default prompt for detailed sentiment analysis.")


    try:
        # Make a real HTTP POST request to your Ollama instance
        data = {
            "model": ollama_model,
            "prompt": prompt,
            "stream": False # We want the full response at once
        }

        # Use basic authentication if username and password are provided
        auth = (username, password) if username and password else None

        # Explicitly set the Origin header
        # We'll use the base ngrok URL as the origin
        origin_url = ollama_url.replace("/api/generate", "") if "/api/generate" in ollama_url else ollama_url
        headers = {
            "Origin": origin_url
        }

        print(f"ℹ️  Ollama Client: Authenticating with Username: {username}, Password provided: {'Yes' if password else 'No'}")

        print(f"ℹ️  Ollama Client: Sending sentiment analysis request to Ollama.")
        # Pass the auth parameter to requests.post
        response = requests.post(ollama_url, json=data, headers=headers, auth=auth) # Pass headers and auth
        print(f"ℹ️  Ollama Client: POST request sent. Status Code: {response.status_code}") # Debug print
        response.raise_for_status() # Raise an exception for bad status codes
        result = response.json()
        print(f"ℹ️  Ollama Client: Received response from Ollama.") # Debug print

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(result.get('response', '').strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Ollama Client: Ollama sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Ollama",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Ollama Client: Ollama returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Ollama",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Ollama Client: Ollama did not return valid JSON. Raw response: {result.get('response', '')}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Ollama",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except requests.exceptions.RequestException as e:
        print(f"⛔ Ollama Client: ERROR communicating with Ollama instance at {ollama_url}: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Ollama Client: An unexpected error occurred during Ollama analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Ollama",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }






def analyze_sentiment_with_gemini(all_content_text, prompt_override=None, symbol="TSLA", company_name="Tesla"):
    """Analyzes the sentiment of a given news item or list of news items using the Gemini API."""
    # symbol and company_name are now parameters with default values

    # Use the prompt_override if provided, otherwise use the default prompt
    if prompt_override:
        prompt = prompt_override
        print("ℹ️  Gemini Client: Using prompt override for sentiment analysis.")
    else:
        # Default prompt for overall sentiment analysis of multiple items
        # Modified to request a detailed breakdown
        prompt = f"""Analyze the following financial news headlines and summaries about {company_name} ({symbol}). Categorize each individual news item's sentiment as POSITIVE, NEUTRAL, or NEUTRAL. Then, provide an overall sentiment for the collection of news and a count or percentage breakdown of the individual sentiments.
Respond in a JSON format like this:
{{
  "overall_sentiment": "POSITIVE|NEUTRAL|NEGATIVE",
  "breakdown": {{
    "positive_count": int,
    "neutral_count": int,
    "negative_count": int,
    "positive_percentage": float,
    "neutral_percentage": float,
    "negative_percentage": float
  }}
}}

News Content:
---
{all_content_text}
---

Sentiment Analysis:"""
        print(f"ℹ️  Gemini Client: Using default prompt for detailed sentiment analysis.")

    try:
        # Configure Gemini API
        # Assumes GOOGLE_API_KEY is stored in Colab secrets
        print("ℹ️  Gemini Client: Attempting to access GOOGLE_API_KEY from Colab secrets.")
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)

        # Initialize the Gemini model
        # Use a suitable model, e.g., 'gemini-1.5-flash-latest' or 'gemini-1.0-pro'
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # You can change the model here

        print(f"ℹ️  Gemini Client: Sending sentiment analysis request to Gemini.")

        # Generate content using the prompt
        response = gemini_model.generate_content(prompt)

        # Attempt to parse the JSON response
        try:
            sentiment_data = json.loads(response.text.strip())
            overall_sentiment = sentiment_data.get('overall_sentiment', '').upper()
            breakdown = sentiment_data.get('breakdown', {})

            if overall_sentiment in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                 print(f"✅ Gemini Client: Gemini sentiment analysis successful. Overall: {overall_sentiment}")
                 return {
                    "symbol": symbol, # Add symbol here
                    "sentiment": overall_sentiment,
                    "source": "Gemini",
                    "headline": "Overall News Sentiment", # Indicate this is overall sentiment
                    "breakdown": breakdown # Include the breakdown
                }
            else:
                print(f"⚠️  Gemini Client: Gemini returned unexpected overall sentiment format: '{overall_sentiment}'. Defaulting to NEUTRAL.")
                return {
                    "symbol": symbol,
                    "sentiment": "NEUTRAL",
                    "source": "Gemini",
                    "headline": "Overall News Sentiment",
                    "breakdown": {} # Return empty breakdown on error
                }

        except json.JSONDecodeError:
            print(f"⚠️  Gemini Client: Gemini did not return valid JSON. Raw response: {response.text.strip()}. Defaulting to NEUTRAL.")
            return {
                "symbol": symbol,
                "sentiment": "NEUTRAL",
                "source": "Gemini",
                "headline": "Overall News Sentiment",
                "breakdown": {} # Return empty breakdown on error
            }


    except TimeoutError as e:
        print(f"⛔ Gemini Client: ERROR: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI. Please ensure you are running directly in the Colab UI. Error: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }
    except Exception as e:
        print(f"⛔ Gemini Client: An unexpected error occurred during Gemini analysis: {e}. Defaulting to NEUTRAL.")
        return {
            "symbol": symbol, # Add symbol here
            "sentiment": "NEUTRAL",
            "source": "Gemini",
            "headline": "Overall News Sentiment", # Indicate this is overall sentiment
            "breakdown": {} # Return empty breakdown on error
        }

# SentimentAgent class definition is in a separate cell (cell fac0357d)
# import json # Import json for parsing the response # Already imported

**Reasoning**:
I have corrected the function signatures. Now I need to re-run the `test_publisher` in cell `6Xt6lVKah7YG` to verify that the sentiment breakdown is included in the output as required by the subtask instructions.



In [50]:
# Assuming all the agent classes from the prototype are defined above

# Added prompt_override as an explicit argument to test_publisher
def test_publisher(Bus, Agent, Topic, prompt_override=None, *agent_args, **agent_kwargs):
    print("--- Testing ---")

    # This list will capture any data our agent publishes
    captured_data = []
    captured_all_content_text = "" # Variable to store the consolidated text

    # A simple listener function
    def data_listener(data):
        nonlocal captured_all_content_text # Allow modification of the outer scope variable
        print(f"Listener caught data: {data}")
        captured_data.append(data)
        # Assuming the all_content_text is part of the data published,
        # or we need to access it from the agent instance.
        # Since the agent analyzes and publishes, the consolidated text is
        # generated before publishing. We need to capture it before it's used
        # in the analyze_sentiment call within run_tick.

    # Subscribe our listener to the event the agent publishes
    Bus.subscribe(Topic, data_listener)

    # Create the agent, passing through other args/kwargs for initialization
    agent = Agent(Bus, *agent_args, **agent_kwargs)

    # Temporarily set the prompt_override attribute on the agent instance for this test run
    # The agent's run_tick method will need to check for this attribute
    if prompt_override is not None:
        agent.prompt_override = prompt_override

    # Modify the run_tick method of the agent instance for testing purposes
    # to capture the all_content_text before analysis and publishing
    original_run_tick = agent.run_tick
    def modified_run_tick():
        nonlocal captured_all_content_text
        # Call the original run_tick
        original_run_tick()
        # After the original run_tick, the agent will have processed news and
        # potentially analyzed sentiment. We need to find a way to access
        # the all_content_text that was generated just before analysis.
        # This is tricky because the agent's internal state might not
        # expose this easily.

        # A simpler approach for testing is to modify the analyze_sentiment call
        # within run_tick to also print the all_content_text.
        pass # We will modify analyze_sentiment instead

    # Modify the analyze_sentiment method of the agent instance
    original_analyze_sentiment = agent.analyze_sentiment
    def modified_analyze_sentiment(all_content_text_arg):
        print(f"--- Consolidated News Text for Analysis ---\n{all_content_text_arg}\n--- End Consolidated News Text ---")
        # Call the original analyze_sentiment
        return original_analyze_sentiment(all_content_text_arg)

    agent.analyze_sentiment = modified_analyze_sentiment


    # Call the agent's main tick function
    agent.run_tick()

    # Verify the results
    # We expect at least one sentiment signal to be published if news is found
    assert len(captured_data) > 0, "Expected at least one sentiment signal, but got none."
    assert captured_data[0]['symbol'] == 'TSLA'
    # Optional: Add checks for sentiment or headline based on prompt_override if applicable
    print("✅ Test Passed: The Agent published its data correctly.\n")

# Run the test
Bus = EventBus()
# --- Update this line to pass your ngrok URL ---
# Replace "YOUR_NGROK_HTTPS_URL" with the https:// URL from ngrok + /api/generate
ngrok_ollama_url = "https://47c47f70c36c.ngrok-free.app/api/generate" # <-- PASTE YOUR CORRECT NGROK URL + /api/generate HERE
username = "hw0531" # <-- Your username here
password = "Aol12345" # <-- Your password here

#Agent = MarketDataAgent(Bus)
#Topic = "NEW_MARKET_DATA"
#Agent = ScheduledEventAgent(Bus)
#Topic = "EVENT_IMMINENT"
Agent = SentimentAgent
Topic = "SENTIMENT_SIGNAL"

# Example of calling test_publisher with prompt_override (replace the string as needed)
test_publisher(
    Bus,
    Agent,
    Topic,
    # prompt_override="Analyze the sentiment of this text: 'TSLA stock is plumetting!' Respond with one word: POSITIVE, NEUTRAL, or NEGATIVE.",
    ollama_url=ngrok_ollama_url,
    use_ollama=True, # Set to False to test Gemini
    ollama_username=username,
    ollama_password=password,
    verbose=True # Set to True for verbose output
)

--- Testing ---
[SentimentAgent]: Fetching news from Yahoo Finance...
✅ SentimentAgent: Found 10 raw news items.
[SentimentAgent]: Analyzing overall sentiment.
--- Consolidated News Text for Analysis ---
{'id': 'b7c0fe9a-ad27-366c-8a8b-ebfe5a980438', 'contentType': 'STORY', 'title': 'Tesla sales in France and Denmark continue downward trend', 'description': '', 'summary': 'Investing.com -- Tesla Inc (NASDAQ:TSLA) car sales in France and Denmark declined in July for the seventh consecutive month.', 'pubDate': '2025-08-01T07:56:00Z', 'displayTime': '2025-08-01T07:56:00Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': None, 'provider': {'displayName': 'Investing.com', 'url': 'https://www.investing.com/'}, 'canonicalUrl': {'url': 'https://finance.yahoo.com/news/tesla-sales-france-denmark-continue-075600825.html', 'site': 'finance', 'region': 'US', 'lang': 'en-US'}, 'clickThroughUrl': {'url': 'https://finance.yahoo.com/news/tesla-sales-france-denmark-continue-0756

## Summary:

### Data Analysis Key Findings

*   The sentiment analysis functions (`analyze_sentiment_with_ollama` and `analyze_sentiment_with_gemini`) were successfully modified to request and parse a detailed sentiment breakdown, including counts and percentages of positive, neutral, and negative news items, in a structured JSON format.
*   The `SentimentAgent` was found to already be compatible with the new sentiment data structure, as it publishes the entire dictionary returned by the sentiment analysis functions, including the `breakdown` key.
*   The `StrategyAgent`'s current logic relies only on the overall sentiment category (POSITIVE/NEGATIVE) and does not require the sentiment percentage breakdown for its existing trading decisions.
*   Testing revealed an initial `TypeError` due to the `SentimentAgent` passing `symbol` and `company_name` arguments that the sentiment analysis functions did not accept; this was corrected by updating the function signatures.
*   While the code was successfully updated to handle and publish the sentiment breakdown, a specific test run showed the language model failing to return the expected JSON format, resulting in an empty `breakdown` dictionary being published in that instance.

### Insights or Next Steps

*   Implement more robust error handling or validation within the sentiment analysis functions to ensure the language model's output adheres to the expected JSON format, or provide clearer feedback when it doesn't.
*   Explore opportunities to enhance the `StrategyAgent` by incorporating the sentiment percentage breakdown to potentially develop more nuanced trading strategies (e.g., trading only when positive sentiment exceeds a certain threshold).
